The following simulator works with any number of gates

In [ ]:
import random
import numpy as np
import math

In [ ]:
l=['0']
from collections import deque
def generate(n):
    q = deque()
    q.append('1')
    for i in range(n):
        front = str(q.popleft())
        q.append(front + '0')
        q.append(front + '1')
        l.append(front)
    return l

In [ ]:
def Y_gate():
  Y=np.array([[0,-1j],[1j,0]])
  return Y

def Z_gate():
  Z=np.array([[1,0],[0,-1]])
  return Z

def S_gate():
  S=np.array([[1,0],[0,1j]])
  return S

def T_gate():
  val=(1+1j)/math.sqrt(2)
  T=np.array([[1,0],[0,val]])
  return T

In [ ]:
def X_gate():
   X = np.array([[0, 1],[1, 0]])
   return X

def Hadamard():
  H = np.array([[1/np.sqrt(2), 1/np.sqrt(2)],[1/np.sqrt(2), -1/np.sqrt(2)]])
  return H

def CNot(total_qubits,target_qubits):
  X = np.array([[0, 1],[1, 0]])
  # Define 2x2 Identity

  I = np.identity(2)
  # Define projection operator |0><0|

  P0x0 = np.array([[1, 0],[0, 0]])

  # Define projection operator |1><1|

  P1x1 = np.array([[0, 0],[0, 1]])
  ans0=[]
  ans1=[]
  for i in range(total_qubits):
    if i==target_qubits[0]:
      ans0.append(P0x0)
      ans1.append(P1x1)
    elif i==target_qubits[1]:
      ans0.append(I)
      ans1.append(X)
    else:
      ans0.append(I)
      ans1.append(I)
  operator0=np.kron(ans0[0],ans0[1])
  operator1=np.kron(ans1[0],ans1[1])
  for i in range(2,total_qubits):
    operator0=np.kron(operator0,ans0[i])
    operator1=np.kron(operator1,ans1[i])
  return operator0+operator1

def get_power(num):
    power=0
    while(num>1):
      num=num/2
      power+=1
    return power

In [ ]:
def get_ground_state(num_qubits):
    # return vector of size 2**num_qubits with all zeroes except first element which is 1
    state_vector=[0]*2**num_qubits
    state_vector[0]=1
    return state_vector

def get_operator(total_qubits, gate_unitary, target_qubits):
    I=np.identity(2)
    if gate_unitary=='cx':
      return CNot(total_qubits,target_qubits)
    elif gate_unitary=='x':
      gate=X_gate()
    elif gate_unitary=='h':
      gate=Hadamard()
    elif gate_unitary=='y':
      gate=Y_gate()
    elif gate_unitary=='z':
      gate=Z_gate()
    elif gate_unitary=='s':
      gate=S_gate()
    elif gate_unitary=='t':
      gate=T_gate()
    ans=[]
    for i in range(total_qubits):
      if i==target_qubits[0]:
        ans.append(gate)
      else:
        ans.append(I)
  #ans[target_qubits[0]]=gate
    op=np.kron(ans[0],ans[1])
    for i in range(2,total_qubits):
      op=np.kron(op,ans[i])
    return op

def run_program(initial_state, program):
    # read program, and for each gate
    #   - calculate matrix operator
    #   - multiply state with operator
    # return final state
    current_state=initial_state
    for i in range(len(program)):
      total_qubits=get_power(len(current_state))
      operator=get_operator(total_qubits,program[i]['gate'],program[i]['target'])
      current_state=np.dot(current_state,operator)
    return current_state

def measure_all(state_vector):
    # choose element from state_vector using weighted random and return it's index
    return

def get_counts(state_vector, num_shots):
    # simply execute measure_all in a loop num_shots times and
    # return object with statistics in following form:
    #   {
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      ...
    #   }
    # (only for elements which occoured - returned from measure_all)
    d={}
    arr=[]
    cnt=0
    n=len(state_vector)
    l=generate(n-1)
    correct=[]
    p=get_power(n)
    for i in range(n):
      if(len(l[i])!=p):
         diff=p-len(l[i])
         correct.append(l[i].rjust(diff + len(l[i]), '0'))
      else:
        correct.append(l[i])
    arr=correct
    probability_distribution=[]
    for i in state_vector:
      probability_distribution.append(i*i)
      #arr.append(cnt)
      cnt+=1
    ans=random.choices(arr,probability_distribution,k=num_shots)
    for i in range(len(arr)):
      d[arr[i]]=ans.count(arr[i])
    counts_nz={}
    for i in d.keys():
      if(d[i]!=0):
        counts_nz[i]=d[i]
    return counts_nz

In [ ]:
my_circuit = [
{ "gate": "h", "target": [0] }, 
{ "gate": "cx", "target": [0, 1] }
]


# Create "quantum computer" with 2 qubits (this is actually just a vector :) )

my_qpu = get_ground_state(2)


# Run circuit

final_state = run_program(my_qpu, my_circuit)


# Read results

counts = get_counts(final_state, 1000)

print(counts)

# Should print something like:
# {
#   "00": 502,
#   "11": 498
# }


{'00': 502, '11': 498}
